In [1]:
import mido
import os
import numpy as np
from scipy import stats
from os import listdir
from os.path import isfile, join
import tensorflow as tf
from tensorflow.contrib import rnn
import time

import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = (12, 8)

In [2]:
TICKS_PER_BEAT = 480
TEMPO = int(mido.bpm2tempo(120))

In [3]:
%run get_training_data.py

### Create training data

In [4]:
def get_training_data(num):
    """Generate training data array for all files in "midis_processed/" directory.
       
    Returns:
        Numpy array of training data.
    """
    training_data = []
    for filename in os.listdir('midis_processed/')[:num]:
        if filename.endswith(".mid"):
            vector = midi_to_vector('midis_processed/' + filename)
            if len(vector) > 50:
                training_data.append(vector)
            else:
                print("Faulty training data: " + filename + ". len(vector)=" + str(len(vector)))
    return np.array(training_data)

In [5]:
training_data = get_training_data(100)

Faulty training data: Adventure_Island_II-Eggplant.mid. len(vector)=0
Faulty training data: Adventure_Island_II-Game_Over.mid. len(vector)=0
Faulty training data: Adventure_Island_II-Victory.mid. len(vector)=0
Faulty training data: Alfred_Chicken_Level_1.mid. len(vector)=0
Faulty training data: alien3sb.mid. len(vector)=0
Faulty training data: alphago.mid. len(vector)=0
Faulty training data: Amagon-Death.mid. len(vector)=0
Faulty training data: Amagon-Start_2.mid. len(vector)=0


In [6]:
training_data = np.array([np.array(x) for x in training_data])
training_data

array([ array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0,

### Parameters

In [7]:
vocab_size = len(training_data[0][0])
vocab_size

1124

In [8]:
batch_size = len(training_data)
batch_size

92

In [ ]:
# Parameters
learning_rate = 0.001
training_iters = 1000
display_step = 1
n_input = 50

# number of units in RNN cell
n_hidden = 512

In [9]:
# tf Graph input
x = tf.placeholder("float", [batch_size, n_input, vocab_size])
y = tf.placeholder("int32", [batch_size, n_input])

# RNN output node weights and biases
softmax_w = tf.Variable(tf.random_normal([n_hidden, vocab_size]))
softmax_b = tf.Variable(tf.random_normal([vocab_size]))

max_grad_norm = 5

In [10]:
# tf Graph generative input
x_generation = tf.placeholder("float", [1, vocab_size])

## Create tensor flow graph

In [11]:
rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden), rnn.BasicLSTMCell(n_hidden)])

In [12]:
x_split = tf.split(x, n_input, 1)
x_split

[<tf.Tensor 'split:0' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:1' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:2' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:3' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:4' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:5' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:6' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:7' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:8' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:9' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:10' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:11' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:12' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:13' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:14' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:15' shape=(92, 1, 1124) dtype=float32>,
 <tf.Tensor 'split:16' shape=(92, 1, 1124) dtype=float32>,
 <tf.Te

In [13]:
x_split = [tf.reshape(a, [batch_size, vocab_size]) for a in x_split]
x_split

[<tf.Tensor 'Reshape:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_1:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_2:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_3:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_4:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_5:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_6:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_7:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_8:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_9:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_10:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_11:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_12:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_13:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_14:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_15:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'Reshape_16:0' shape=(92, 1124) dtype=fl

In [14]:
outputs, states = rnn.static_rnn(rnn_cell, x_split, dtype=tf.float32)
outputs

[<tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_2:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_5:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_8:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_11:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_14:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_17:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_20:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_23:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_26:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_

In [15]:
logits = [tf.nn.xw_plus_b(output, softmax_w, softmax_b) for output in outputs]
logits

[<tf.Tensor 'xw_plus_b:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_1:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_2:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_3:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_4:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_5:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_6:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_7:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_8:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_9:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_10:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_11:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_12:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_13:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_14:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_15:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus

In [16]:
logits = tf.reshape(logits, [batch_size, n_input, vocab_size])
logits

<tf.Tensor 'Reshape_50:0' shape=(92, 50, 1124) dtype=float32>

In [17]:
loss = tf.contrib.seq2seq.sequence_loss(logits, y, weights=tf.ones([batch_size, n_input], dtype=tf.float32), # check this
                                        average_across_timesteps=False, average_across_batch=True)
loss

<tf.Tensor 'sequence_loss/truediv:0' shape=(50,) dtype=float32>

In [18]:
cost = tf.reduce_sum(loss)
cost

<tf.Tensor 'Sum:0' shape=() dtype=float32>

In [19]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer

<tf.Operation 'RMSProp' type=NoOp>

## Run tensor flow session

In [20]:
session = tf.Session()

In [21]:
def get_midi_xy(midi_file_number):
    offset = np.random.randint(0, len(training_data[midi_file_number]) - n_input - 1)
    midi_x = training_data[midi_file_number][offset:offset+n_input]
    midi_y = training_data[midi_file_number][1 + offset:1 + offset+n_input].dot(range(vocab_size)).reshape(1, n_input)
    return midi_x, midi_y

In [22]:
def get_batch():
    batch_x = []
    batch_y = []
    for i in range(batch_size):
        midi_x, midi_y = get_midi_xy(i%len(training_data))
        batch_x.append(midi_x)
        batch_y.append(midi_y)
    batch_y = np.reshape(batch_y, [batch_size, n_input])
    return batch_x, batch_y

In [ ]:
def seconds_to_minsecstr(seconds):
    minutes = seconds//60
    seconds -= 60*minutes
    if minutes < 10: minutes = "0" + str(minutes)
    if seconds < 10: seconds = "0" + str(seconds)
    return str(minutes) + ":" + str(seconds)

In [ ]:
training_loss_list = []

init = tf.global_variables_initializer()
session.run(init)
step = 0
loss_total = 0

start_time = time.time()

while step < training_iters:
    for i in range(len(training_data)):
        batch_x, batch_y = get_batch()
        
        _, loss = session.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
        loss_total += loss
        if (step+1) % display_step == 0:
            print("Iter=" + str(step+1) + ", Average Loss={:.6f}, Time Elapsed={}".format(loss_total/display_step, 
                 seconds_to_minsecstr(int(time.time()-start_time))))
            loss_total = 0
        training_loss_list.append(loss_total/display_step)
        step += 1

Iter=1, Average Loss=376.194122, Time Elapsed=00:00.07
Iter=2, Average Loss=362.741852, Time Elapsed=00:00.14
Iter=3, Average Loss=354.501862, Time Elapsed=00:00.20
Iter=4, Average Loss=349.700958, Time Elapsed=00:00.26
Iter=5, Average Loss=341.575897, Time Elapsed=00:00.34
Iter=6, Average Loss=339.359161, Time Elapsed=00:00.42
Iter=7, Average Loss=333.342194, Time Elapsed=00:00.49
Iter=8, Average Loss=335.661804, Time Elapsed=00:00.56
Iter=9, Average Loss=334.497803, Time Elapsed=00:01.03
Iter=10, Average Loss=331.893250, Time Elapsed=00:01.09
Iter=11, Average Loss=343.635162, Time Elapsed=00:01.15
Iter=12, Average Loss=331.490845, Time Elapsed=00:01.21
Iter=13, Average Loss=323.286560, Time Elapsed=00:01.27
Iter=14, Average Loss=322.483215, Time Elapsed=00:01.34
Iter=15, Average Loss=329.100159, Time Elapsed=00:01.40
Iter=16, Average Loss=357.856598, Time Elapsed=00:01.47
Iter=17, Average Loss=344.309082, Time Elapsed=00:01.53
Iter=18, Average Loss=329.769836, Time Elapsed=00:01.59
I

Iter=147, Average Loss=271.929382, Time Elapsed=00:15.07
Iter=148, Average Loss=267.983215, Time Elapsed=00:15.11
Iter=149, Average Loss=270.830139, Time Elapsed=00:15.16
Iter=150, Average Loss=261.905060, Time Elapsed=00:15.20
Iter=151, Average Loss=260.138184, Time Elapsed=00:15.25
Iter=152, Average Loss=267.838684, Time Elapsed=00:15.30
Iter=153, Average Loss=268.758636, Time Elapsed=00:15.34
Iter=154, Average Loss=265.636505, Time Elapsed=00:15.39
Iter=155, Average Loss=269.743347, Time Elapsed=00:15.43
Iter=156, Average Loss=267.147278, Time Elapsed=00:15.47
Iter=157, Average Loss=262.826935, Time Elapsed=00:15.52
Iter=158, Average Loss=263.173523, Time Elapsed=00:15.56
Iter=159, Average Loss=261.791107, Time Elapsed=00:16.01
Iter=160, Average Loss=266.434387, Time Elapsed=00:16.05
Iter=161, Average Loss=262.171021, Time Elapsed=00:16.10
Iter=162, Average Loss=263.764648, Time Elapsed=00:16.14
Iter=163, Average Loss=260.802368, Time Elapsed=00:16.19
Iter=164, Average Loss=271.5886

Iter=291, Average Loss=248.247147, Time Elapsed=00:26.42
Iter=292, Average Loss=248.297913, Time Elapsed=00:26.47
Iter=293, Average Loss=243.892624, Time Elapsed=00:26.52
Iter=294, Average Loss=240.213303, Time Elapsed=00:26.57
Iter=295, Average Loss=245.460632, Time Elapsed=00:27.02
Iter=296, Average Loss=246.202515, Time Elapsed=00:27.07
Iter=297, Average Loss=242.663040, Time Elapsed=00:27.12
Iter=298, Average Loss=241.145844, Time Elapsed=00:27.17
Iter=299, Average Loss=240.247543, Time Elapsed=00:27.22
Iter=300, Average Loss=241.859985, Time Elapsed=00:27.28
Iter=301, Average Loss=245.856567, Time Elapsed=00:27.33
Iter=302, Average Loss=237.746826, Time Elapsed=00:27.39
Iter=303, Average Loss=238.792557, Time Elapsed=00:27.44
Iter=304, Average Loss=239.323929, Time Elapsed=00:27.50
Iter=305, Average Loss=239.649414, Time Elapsed=00:27.56
Iter=306, Average Loss=243.395111, Time Elapsed=00:28.03
Iter=307, Average Loss=247.971237, Time Elapsed=00:28.09
Iter=308, Average Loss=240.9210

Iter=435, Average Loss=231.130051, Time Elapsed=00:40.37
Iter=436, Average Loss=238.998199, Time Elapsed=00:40.42
Iter=437, Average Loss=235.905365, Time Elapsed=00:40.48
Iter=438, Average Loss=233.194000, Time Elapsed=00:40.53
Iter=439, Average Loss=229.319366, Time Elapsed=00:40.59
Iter=440, Average Loss=231.224472, Time Elapsed=00:41.04
Iter=441, Average Loss=229.477768, Time Elapsed=00:41.10
Iter=442, Average Loss=234.943390, Time Elapsed=00:41.15
Iter=443, Average Loss=229.257385, Time Elapsed=00:41.21
Iter=444, Average Loss=233.090302, Time Elapsed=00:41.26
Iter=445, Average Loss=233.243896, Time Elapsed=00:41.32
Iter=446, Average Loss=231.957306, Time Elapsed=00:41.37
Iter=447, Average Loss=232.095749, Time Elapsed=00:41.43
Iter=448, Average Loss=234.358643, Time Elapsed=00:41.48
Iter=449, Average Loss=227.694839, Time Elapsed=00:41.54
Iter=450, Average Loss=226.943665, Time Elapsed=00:41.59
Iter=451, Average Loss=231.729324, Time Elapsed=00:42.05
Iter=452, Average Loss=234.8978

Iter=579, Average Loss=228.027359, Time Elapsed=00:53.56
Iter=580, Average Loss=226.161163, Time Elapsed=00:54.02
Iter=581, Average Loss=225.252792, Time Elapsed=00:54.08
Iter=582, Average Loss=227.118866, Time Elapsed=00:54.13
Iter=583, Average Loss=226.110214, Time Elapsed=00:54.19
Iter=584, Average Loss=227.248688, Time Elapsed=00:54.24
Iter=585, Average Loss=222.226776, Time Elapsed=00:54.30
Iter=586, Average Loss=223.761871, Time Elapsed=00:54.36
Iter=587, Average Loss=222.828720, Time Elapsed=00:54.41
Iter=588, Average Loss=223.399033, Time Elapsed=00:54.47
Iter=589, Average Loss=223.615295, Time Elapsed=00:54.53
Iter=590, Average Loss=223.592209, Time Elapsed=00:54.58
Iter=591, Average Loss=226.668304, Time Elapsed=00:55.05
Iter=592, Average Loss=223.903152, Time Elapsed=00:55.10
Iter=593, Average Loss=226.245117, Time Elapsed=00:55.16
Iter=594, Average Loss=224.010635, Time Elapsed=00:55.21
Iter=595, Average Loss=228.018204, Time Elapsed=00:55.27
Iter=596, Average Loss=225.5411

Iter=723, Average Loss=218.541016, Time Elapsed=01:07.28
Iter=724, Average Loss=220.081375, Time Elapsed=01:07.34
Iter=725, Average Loss=221.079956, Time Elapsed=01:07.40
Iter=726, Average Loss=219.372391, Time Elapsed=01:07.45
Iter=727, Average Loss=217.526016, Time Elapsed=01:07.51
Iter=728, Average Loss=215.953827, Time Elapsed=01:07.56
Iter=729, Average Loss=221.473404, Time Elapsed=01:08.02
Iter=730, Average Loss=221.898178, Time Elapsed=01:08.08
Iter=731, Average Loss=218.692917, Time Elapsed=01:08.13
Iter=732, Average Loss=217.931824, Time Elapsed=01:08.19
Iter=733, Average Loss=223.784531, Time Elapsed=01:08.24
Iter=734, Average Loss=224.565750, Time Elapsed=01:08.30
Iter=735, Average Loss=218.927063, Time Elapsed=01:08.36
Iter=736, Average Loss=220.960571, Time Elapsed=01:08.41
Iter=737, Average Loss=217.833542, Time Elapsed=01:08.47
Iter=738, Average Loss=218.868317, Time Elapsed=01:08.53
Iter=739, Average Loss=218.807739, Time Elapsed=01:08.58
Iter=740, Average Loss=221.2318

Iter=867, Average Loss=218.318283, Time Elapsed=01:20.40
Iter=868, Average Loss=217.380737, Time Elapsed=01:20.45
Iter=869, Average Loss=215.262405, Time Elapsed=01:20.50
Iter=870, Average Loss=217.532608, Time Elapsed=01:20.56
Iter=871, Average Loss=217.350082, Time Elapsed=01:21.01
Iter=872, Average Loss=217.248596, Time Elapsed=01:21.06
Iter=873, Average Loss=214.500946, Time Elapsed=01:21.12
Iter=874, Average Loss=219.085236, Time Elapsed=01:21.17
Iter=875, Average Loss=214.015137, Time Elapsed=01:21.22
Iter=876, Average Loss=217.628204, Time Elapsed=01:21.28
Iter=877, Average Loss=214.242203, Time Elapsed=01:21.33
Iter=878, Average Loss=217.413147, Time Elapsed=01:21.38
Iter=879, Average Loss=216.772751, Time Elapsed=01:21.44
Iter=880, Average Loss=216.121307, Time Elapsed=01:21.49
Iter=881, Average Loss=216.531601, Time Elapsed=01:21.55
Iter=882, Average Loss=215.785477, Time Elapsed=01:22.00
Iter=883, Average Loss=210.798950, Time Elapsed=01:22.05
Iter=884, Average Loss=216.8793

Iter=1011, Average Loss=210.811737, Time Elapsed=01:33.30
Iter=1012, Average Loss=208.783432, Time Elapsed=01:33.35
Iter=1013, Average Loss=210.519302, Time Elapsed=01:33.40
Iter=1014, Average Loss=212.642624, Time Elapsed=01:33.46
Iter=1015, Average Loss=217.943512, Time Elapsed=01:33.51
Iter=1016, Average Loss=212.307617, Time Elapsed=01:33.57
Iter=1017, Average Loss=209.930954, Time Elapsed=01:34.02
Iter=1018, Average Loss=210.300049, Time Elapsed=01:34.08
Iter=1019, Average Loss=209.427246, Time Elapsed=01:34.13
Iter=1020, Average Loss=211.665771, Time Elapsed=01:34.19
Iter=1021, Average Loss=211.229080, Time Elapsed=01:34.24
Iter=1022, Average Loss=214.838486, Time Elapsed=01:34.30
Iter=1023, Average Loss=214.283218, Time Elapsed=01:34.35
Iter=1024, Average Loss=215.118210, Time Elapsed=01:34.40
Iter=1025, Average Loss=210.535919, Time Elapsed=01:34.46
Iter=1026, Average Loss=215.612518, Time Elapsed=01:34.51
Iter=1027, Average Loss=212.375931, Time Elapsed=01:34.56
Iter=1028, Ave

Iter=1153, Average Loss=209.067474, Time Elapsed=01:46.09
Iter=1154, Average Loss=211.886948, Time Elapsed=01:46.15
Iter=1155, Average Loss=206.745285, Time Elapsed=01:46.20
Iter=1156, Average Loss=208.245621, Time Elapsed=01:46.25
Iter=1157, Average Loss=206.743927, Time Elapsed=01:46.31
Iter=1158, Average Loss=208.442459, Time Elapsed=01:46.36
Iter=1159, Average Loss=204.427719, Time Elapsed=01:46.41
Iter=1160, Average Loss=205.574463, Time Elapsed=01:46.47
Iter=1161, Average Loss=207.439285, Time Elapsed=01:46.52
Iter=1162, Average Loss=210.539917, Time Elapsed=01:46.57
Iter=1163, Average Loss=204.789429, Time Elapsed=01:47.03
Iter=1164, Average Loss=206.946686, Time Elapsed=01:47.08
Iter=1165, Average Loss=204.476089, Time Elapsed=01:47.13
Iter=1166, Average Loss=204.675110, Time Elapsed=01:47.19
Iter=1167, Average Loss=205.370651, Time Elapsed=01:47.24
Iter=1168, Average Loss=207.449966, Time Elapsed=01:47.29
Iter=1169, Average Loss=207.923431, Time Elapsed=01:47.35
Iter=1170, Ave

## Generate music from trained model

In [ ]:
N_GENERATED_NOTE_VECTORS = 1000

In [ ]:
outputs_generation, states_generation = rnn.static_rnn(rnn_cell, [x_generation], dtype=tf.float32)
outputs_generation

In [ ]:
logits_generation = tf.nn.xw_plus_b(outputs_generation[0], softmax_w, softmax_b)
logits_generation

In [ ]:
seed_note = np.zeros(vocab_size)
seed_note[0] = 1
seed_note = np.reshape(seed_note, [1, vocab_size])

generated_music = [seed_note]
for i in range(N_GENERATED_NOTE_VECTORS):
    onehot_pred = session.run(logits_generation, feed_dict={x_generation: generated_music[-1]})

    next_note = np.zeros(vocab_size)
    next_note[np.argmax(onehot_pred, 1)] = 1
    next_note = np.reshape(next_note,  [1, vocab_size])
    
    generated_music.append(next_note)

# Convert generated music to midi

In [ ]:
def get_track_sequences(vector_sequence):
    """Get note sequences for each track from vector sequence output from RNN.

    Args:
       vector_sequence: List of one-hot vectors containing 128 * NUM_TRACKS note_on events, 128 * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 10 ms each.
       
    Returns:
        List of track vector sequences.
    """
    track_sequences = [[] for _ in range(NUM_TRACKS)]
    start_time = 0
    for vector in vector_sequence:
        index = vector.index(1)
        # vector is a time event.
        if index >= NUM_MIDI_PITCHES * 2 * NUM_TRACKS:
            time = (index - NUM_MIDI_PITCHES * 2 * NUM_TRACKS + 1) * TIMESHIFT_LENGTH
            start_time += time
        else:
            # vector is a note_off event.
            if index >= NUM_MIDI_PITCHES * NUM_TRACKS:
                note_type = 'note_off'
                index -= NUM_MIDI_PITCHES * NUM_TRACKS
            # vector is a note_on event.
            else:
                note_type = 'note_on'
            track_num = int(index / NUM_MIDI_PITCHES)
            note = index % NUM_MIDI_PITCHES
            track_sequences[track_num].append({"type": note_type, "note": note, "start_time": start_time})
            time_delay = 0
            
    return track_sequences
    

In [ ]:
def get_midi_from_vector_sequence(track_sequences):
    """Get midi from track sequences.

    Args:
       track_sequences: List of vector sequences for each track.
       
    Returns:
        MidiFile object.
    """
    mid = mido.MidiFile()
    tracks = [mido.MidiTrack() for _ in range(NUM_TRACKS)]
    mid.tracks.extend(tracks)

    for i, ts in enumerate(track_sequences):
        prev_start_time = 0
        for event in ts:
            time = event["start_time"] - prev_start_time
            prev_start_time = event["start_time"]
            # I picked a random number for velocity.
            mid.tracks[i].append(mido.Message(event["type"], note=event["note"], velocity=50,
                                              time=int(mido.second2tick(time, TICKS_PER_BEAT, TEMPO))))
    
    return mid

In [ ]:
track_sequence = get_track_sequences([list(x[0]) for x in generated_music])
track_sequence

In [ ]:
midi = get_midi_from_vector_sequence(track_sequence)
midi

# Save midi

In [ ]:
midi.save('generated_music.mid')